## Environnement setup

In [2]:
import os
import math

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [3]:
# Setup plots
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8
%config InlineBackend.figure_format = 'retina'

In [4]:
import torch

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 1.10.1+cu102
No GPU :(


## Model

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, n_features, kernel, stride=1, padding='same'):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(n_features, n_features, kernel, padding=padding, stride=stride)
        self.norm1 = nn.BatchNorm2d(n_features)
        self.conv2 = nn.Conv2d(n_features, n_features, kernel, padding=padding, stride=stride)
        self.norm2 = nn.BatchNorm2d(n_features)
    
    def forward(self, x):
        y = self.conv1(x)
        y = self.norm1(y)
        y = F.ReLU(y)
        y = self.conv2(y)
        y = self.norm2(y)
        x += y
        return F.ReLU(x)
        

class AlphaGoNN(nn.Module):
    def __init__(self, dim_board=81, input_dim=15, n_features=64, kernel=3, n_blocks=5, stride=1, padding='same'):
        super(AlphaGoNN, self).__init__()
        self.input_conv = nn.Conv2d(input_dim, n_features, kernel, padding=padding, stride=stride)
        self.input_norm = nn.BatchNorm2d(n_features)
        
        self.bolcks = [ConvBlock(n_features, kernel, padding=padding, stride=stride) for i in range(n_blocks)]
        
        self.policy_conv = nn.Conv2d(n_features, 2, 1, padding=padding, stride=stride)
        self.policy_norm = nn.BatchNorm2d(2)
        self.policy_linear = nn.Linear(2 * dim_board, dim_board + 1)
        
        self.value_conv = nn.Conv2d(n_features, 1, 1, padding=padding, stride=stride)
        self.value_norm = nn.BatchNorm2d(1)
        self.value_linear1 = nn.Linear(dim_board, n_features)
        self.value_linear2 = nn.Linear(n_features, 1)
    
    def _policy(self, x):
        x = self.policy_conv(x)
        x = self.policy_norm(x)
        x = F.ReLU(x)
        x = self.policy_linear(x)
        return x

    def _value(self, x):
        x = self.value_conv(x)
        x = self.value_norm(x)
        x = F.ReLU(x)
        x = self.value_linear1(x)
        x = F.ReLU(x)
        x = self.value_linear2(x)
        x = F.tanh(x)
        return x
    
    def forward(self, x):
        x = self.input_conv(x)
        x = self.input_norm(x)
        x = F.ReLU(x)
        
        for block in self.blocks:
            x = block(x)
        
        p = self._policy(x)
        v = self._value(x)
        return p, v

In [6]:
# Model
model = AlphaGoNN()

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()

# Optimizer
learning_rate = 0.0002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)